In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt
from medcat.cat import CAT
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

C:\Users\BILAL\AppData\Roaming\Python\Python37\site-packages\medcat\cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BILAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install nltk


In [2]:
# Load model pack and Create CAT - the main class from medcat used for concept annotation

model_pack_path = 'medcat/mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5'
cat = CAT.load_model_pack(model_pack_path)

Found an existing unziped model pack at: medcat\mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5, the provided zip will not be touched.
{
  "Model ID": "25be3857ba34bdd5",
  "Last Modifed On": "16 March 2022",
  "History (from least to most recent)": [
    "a474096eb4566638",
    "009617d7ff372682"
  ],
  "Description": "SNOMED INT enriched with UMLS and trained unsupervised on MIMIC-III",
  "Source Ontology": "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z",
  "Location": "MedCAT Rosalind machine, available for public download from https://github.com/CogStack/MedCAT",
  "MetaCAT models": {
    "Status": "Detects is a concept Affirmed or Negated/Hypothetical"
  },
  "Basic CDB Stats": {
    "Number of concepts": 354448,
    "Number of names": 2049216,
    "Number of concepts that received training": 29674,
    "Number of seen training examples in total": 20585988,
    "Average training examples per concept": 693.7382220125362
  },
  "Performance": {
    "ner": {},
    "meta"

In [2]:
test_notes = pd.read_csv('1000_testing_notes_with_smallest_size.csv')
test_notes

,Unnamed: 0,hadm_id,text,text_length
0,0,20019282,\nName: ___ Unit No: ___\n...,2480
1,1,20022343,\nName: ___ Unit No: _...,3001
2,2,20025548,\nName: ___ Unit No: ___\n...,2961
3,3,20027416,\nName: ___ Unit No: ___\n...,1059
4,4,20028418,\nName: ___ Unit No: ___\n \...,1091
...,...,...,...,...
1010,1010,29980814,\nName: ___ Unit No: _...,3329
1011,1011,29980930,\nName: ___ Unit No: ___...,1094
1012,1012,29987299,\nName: ___ Unit No: ___\...,2595
1013,1013,29988390,\nName: ___ Unit No: ...,3013


In [3]:
test_notes.drop(columns = ['Unnamed: 0'], inplace = True)

In [5]:
unique_hadm_notes = test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')


In [6]:
unique_hadm_notes

,hadm_id,text
0,20019282,\nName: ___ Unit No: ___\n...
1,20022343,\nName: ___ Unit No: _...
2,20025548,\nName: ___ Unit No: ___\n...
3,20027416,\nName: ___ Unit No: ___\n...
4,20028418,\nName: ___ Unit No: ___\n \...
...,...,...
1010,29980814,\nName: ___ Unit No: _...
1011,29980930,\nName: ___ Unit No: ___...
1012,29987299,\nName: ___ Unit No: ___\...
1013,29988390,\nName: ___ Unit No: ...


In [7]:
unique_hadm_notes = unique_hadm_notes.reset_index(drop = True)

In [8]:
unique_hadm_notes.text[0]

' \nName:  ___                 Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   M\n \nService: ORTHOPAEDICS\n \nAllergies: \nerythromycin base\n \nAttending: ___.\n \nChief Complaint:\nLeft ankle fracture\n \nMajor Surgical or Invasive Procedure:\nORIF left ankle ___\n \nHistory of Present Illness:\n___ yr old man fall down 3 stairs while holding lumbar at \nconstruction\njob, resulting in immediate L ankle pain. ED ankle fracture, \nsurgery performed\n\n \nPast Medical History:\nPrevious R ankle fracture\nRLE cellulitis\nOsteoarthritis\n\n \nSocial History:\n___\nFamily History:\nN/A\n \nPhysical Exam:\nAOXO3\nsplint intact, positive CSM\nPain well controlled, no N/V\n\n \nPertinent Results:\n___ 08:28PM   WBC-9.5 RBC-4.49* HGB-13.7 HCT-41.0 MCV-91 \nMCH-30.5 MCHC-33.4 RDW-13.8 RDWSD-46.7*\n___ 08:28PM   GLUCOSE-96 UREA N-13 CREAT-0.7 SODIUM-145 \nPOTASSIUM-4.1 CHLORIDE-108 TOTAL CO2-24 ANION GAP-13\n___ 08:28PM   CA

In [9]:
medical_stopwords = stopwords.words("english")


medical_stopwords.extend(['speaking', 'none', 'time', 'flush'])

def process_clinical_note(clinical_note):
    # Define the sections to remove
    sections_to_remove = [
        "Name:",
        "Unit No:",
        "Admission Date:",
        "Discharge Date:",
        "Date of Birth:",
        "Sex:",
        "Service:",
        "Allergies:",
        "Attending:",
        "Past Medical History:",
        "Social History:",
        "Family History:",
        "Vitals:",
        "Pertinent Results:",
        "Medications on Admission:",
        "Discharge Medications:",
        "Discharge Disposition:",
        "Discharge Condition:",
        "Discharge Instructions:",
        "Followup Instructions:"
    ]

    # Split the clinical note into lines
    lines = clinical_note.split('\n')

    # Initialize the processed note
    processed_note = []

    # Flag to exclude lines within unwanted sections
    exclude_section = False

    # Iterate through the lines and filter unwanted sections
    for line in lines:
        if any(section in line for section in sections_to_remove):
            exclude_section = True
        elif line.strip() == "":
            # Empty lines separate sections, so reset the flag
            exclude_section = False

        if not exclude_section:
            processed_note.append(line)

    # Join the lines to create the final note
    final_note = '\n '.join(processed_note)
    
    sections_to_remove = [
        r'chief complaint',
        r'history of present illness',
        r'Major Surgical or Invasive Procedure',
        r'physical exam',
        r'brief hospital course',
        r'Discharge',
        
        r'completed by',
    ]
    
    for pattern in sections_to_remove:
        final_note = re.sub(pattern, '', final_note, flags=re.IGNORECASE)

    # Define patterns to identify negations
    negation_patterns = [
        r'no\s+\w+',
        r'not\s+\w+',
        r'did\s+not\s+have\s+\w+'
    ]
    
    # Filter out sentences with negations
    sentences = [sentence for sentence in final_note.split('\n') if not any(re.search(pattern, sentence, re.IGNORECASE) for pattern in negation_patterns)]

    # Remove keys and special characters
    cleaned_note = re.sub(r'\w+:', '', '\n'.join(sentences), flags=re.IGNORECASE)  # Remove keys (case-insensitive)
    cleaned_note = re.sub(r'[^a-zA-Z\s]', '', cleaned_note)  # Remove special characters
    # Tokenize the note into sentences based on '\n'
    sentences = [sentence.strip() for sentence in cleaned_note.split('\n') if sentence.strip()]

    # Remove stop words and empty sentences
    sentences = [
        ' '.join(word for word in sentence.split() if word.lower() not in medical_stopwords)
        for sentence in sentences
    ]
    sentences = [item for item in sentences if item != '']
    
    final_text = '. '.join(sentence for sentence in sentences)
    return final_text

In [10]:
process_clinical_note(unique_hadm_notes.text[0])

'Left ankle fracture. ORIF left ankle. yr old man fall stairs holding lumbar. construction. job resulting immediate L ankle pain ED ankle fracture. surgery performed. AOXO. splint intact positive CSM. Taken operating room underwent surgical fixation ankle. well controlled Cleared go home without services. left ankle fracture sp ORIF'

In [11]:
unique_hadm_notes['text'] = unique_hadm_notes['text'].apply(process_clinical_note)
unique_hadm_notes

,hadm_id,text
0,20019282,Left ankle fracture. ORIF left ankle. yr old m...
1,20022343,Preterm labor. repeat low transverse cesarean ...
2,20025548,contractions. SVD. Pt arrives co painful contr...
3,20027416,neck cancer. direct laryngoscopy robotic hemig...
4,20028418,admission chemo. Pt tolerated desens without a...
...,...,...
1010,29980814,Postpartum incision check. partial opening pac...
1011,29980930,elective chemotherapy. Pt presented elective c...
1012,29987299,attach. BLOOD WBC RBC Hgb Hct. MCV MCH MCHC RD...
1013,29988390,acute cholecystitis. laparoscopic cholecystect...


In [12]:
icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}



In [13]:
i = 0
for index, row in unique_hadm_notes.iterrows():
    i +=1
    
    if i%100 ==0:
        print(f'{i} notes have been processed!', '\n')
        
        
    single_note = row['text']
    hadm_id = row['hadm_id']

    # Get the entities from the text
    all_entities = cat.get_entities(single_note)

    icd_codes = set()  # Using a set to automatically remove duplicates

    for entity_id, entity_data in all_entities['entities'].items():
        icd10 = entity_data.get('icd10', [])
        for code in icd10:
            if code:  # Check if the code is not an empty string
                icd_codes.add(code)

    icd10_data['hadm_id'].append(hadm_id)
    icd10_data['icd10_codes'].append(list(icd_codes))



100 notes have been processed! 

200 notes have been processed! 

300 notes have been processed! 

400 notes have been processed! 

500 notes have been processed! 

600 notes have been processed! 

700 notes have been processed! 

800 notes have been processed! 

900 notes have been processed! 

1000 notes have been processed! 



In [14]:
# Create a DataFrame from the processed data
df_icd10 = pd.DataFrame(icd10_data)
df_icd10

,hadm_id,icd10_codes
0,20019282,"[M79.67, S82.80]"
1,20022343,"[O60.0, O32.1, R52.9, L53.9]"
2,20025548,"[A09.9, R10.1, K56.6, R52.9, R11]"
3,20027416,"[C76.0, R22.1, C80.9]"
4,20028418,[D39.1]
...,...,...
1010,29980814,"[R76.1, E16.2, T14.0, R68.8, R58, R79.8, R06.0..."
1011,29980930,[]
1012,29987299,"[D61.9, R31, D47.1]"
1013,29988390,"[N23, K81.0, K81.9, K80.5, R10.4, D72.8, R68.8..."


In [15]:
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3] + '.'
    return code

In [16]:
df_icd10['icd10_codes'] = df_icd10['icd10_codes'].apply(lambda lst: [reformat(x) for x in lst])
df_icd10

,hadm_id,icd10_codes
0,20019282,"[M79., S82.]"
1,20022343,"[O60., O32., R52., L53.]"
2,20025548,"[A09., R10., K56., R52., R11.]"
3,20027416,"[C76., R22., C80.]"
4,20028418,[D39.]
...,...,...
1010,29980814,"[R76., E16., T14., R68., R58., R79., R06., D59..."
1011,29980930,[]
1012,29987299,"[D61., R31., D47.]"
1013,29988390,"[N23., K81., K81., K80., R10., D72., R68., R50..."


In [60]:
df_icd10.to_csv('test_set_mapped_icd_hadmids_1k.csv')

unique_hadm_notes.to_csv('test_set_unique_notes_hadmids_1k.csv')


In [17]:
df_icd10.dtypes

hadm_id         int64
icd10_codes    object
dtype: object

In [18]:
#Make this on category level:
df_icd10

,hadm_id,icd10_codes
0,20019282,"[M79., S82.]"
1,20022343,"[O60., O32., R52., L53.]"
2,20025548,"[A09., R10., K56., R52., R11.]"
3,20027416,"[C76., R22., C80.]"
4,20028418,[D39.]
...,...,...
1010,29980814,"[R76., E16., T14., R68., R58., R79., R06., D59..."
1011,29980930,[]
1012,29987299,"[D61., R31., D47.]"
1013,29988390,"[N23., K81., K81., K80., R10., D72., R68., R50..."


In [19]:
df_icd10.drop_duplicates(subset = ['hadm_id'], inplace = True)

In [8]:
icd_codes_with_smallest_size_notes = pd.read_csv('icd_codes_for_1000_testing_notes_with_smallest_size.csv')
icd_codes_with_smallest_size_notes.drop(columns = ['Unnamed: 0'], inplace = True)
icd_codes_with_smallest_size_notes

,hadm_id,icd_code
0,20019282,S82.
1,20019282,Z72.
2,20019282,W10.
3,20019282,Y92.
4,20022343,O60.
...,...,...
8580,29990058,Z30.
8581,29990058,Z23.
8582,29990058,Z86.
8583,29990058,K21.


In [9]:
icd_codes_for_100_testing_notes = pd.merge(sample_notes_100,icd_codes_with_smallest_size_notes, how='inner', left_on = 'hadm_id', right_on = 'hadm_id' )

In [11]:
icd_codes_for_100_testing_notes.drop(columns = ['text'], inplace = True)

In [13]:
icd_codes_for_100_testing_notes.to_csv('icd_codes_for_100_testing_notes.csv')

In [14]:
icd_codes_for_100_testing_notes

,hadm_id,text_length,icd_code
0,20019282,2480,S82.
1,20019282,2480,Z72.
2,20019282,2480,W10.
3,20019282,2480,Y92.
4,20022343,3001,O60.
...,...,...,...
892,20947071,2151,J44.
893,20947071,2151,E11.
894,20947071,2151,Z78.
895,20947071,2151,R60.


In [15]:
sample_notes_100

,hadm_id,text,text_length
0,20019282,\nName: ___ Unit No: ___\n...,2480
1,20022343,\nName: ___ Unit No: _...,3001
2,20025548,\nName: ___ Unit No: ___\n...,2961
3,20027416,\nName: ___ Unit No: ___\n...,1059
4,20028418,\nName: ___ Unit No: ___\n \...,1091
...,...,...,...
95,20919508,\nName: ___ Unit No: ___\n \nAd...,3484
96,20920499,Name: ___ Unit No: ___\n\nDate o...,3320
97,20927529,\nName: ___ Unit No: ___...,3331
98,20928751,\nName: ___ Unit No: ___...,3097


In [21]:
final_notes_with_icd_codes = pd.merge(unique_hadm_notes, icd_codes_with_smallest_size_notes, on='hadm_id', how='left')
final_notes_with_icd_codes

,hadm_id,text,icd_code
0,20019282,Left ankle fracture. ORIF left ankle. yr old m...,S82.
1,20019282,Left ankle fracture. ORIF left ankle. yr old m...,Z72.
2,20019282,Left ankle fracture. ORIF left ankle. yr old m...,W10.
3,20019282,Left ankle fracture. ORIF left ankle. yr old m...,Y92.
4,20022343,Preterm labor. repeat low transverse cesarean ...,O60.
...,...,...,...
8580,29990058,induction labor. primary cesarean delivery. Af...,Z30.
8581,29990058,induction labor. primary cesarean delivery. Af...,Z23.
8582,29990058,induction labor. primary cesarean delivery. Af...,Z86.
8583,29990058,induction labor. primary cesarean delivery. Af...,K21.


In [69]:

unique_icd_codes = set(final_notes_with_icd_codes['icd_code']) #.union(set(icd_code for icd_list in df_icd10['icd10_codes'] for icd_code in icd_list))



In [70]:
len(unique_icd_codes)

769

In [71]:
final_notes_with_icd_codes.icd_code.nunique()

769

In [72]:
# Step 3: Initialize TP, FP, FN, TN counts
counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [73]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%100 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = df_icd10[df_icd10['icd10_codes'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    counts[icd_code] = (len(TP), len(FP), len(FN), TN)

100 codes are done!
200 codes are done!
300 codes are done!
400 codes are done!
500 codes are done!
600 codes are done!
700 codes are done!


In [74]:
counts

{'K13.': (0, 56, 1, 0),
 'N11.': (0, 0, 1, 0),
 'N97.': (1, 0, 0, 0),
 'S31.': (0, 2, 2, 0),
 'D62.': (1, 0, 30, 0),
 'L90.': (0, 3, 1, 0),
 'K31.': (1, 3, 6, 0),
 'C91.': (0, 0, 6, 0),
 'W25.': (0, 0, 3, 0),
 'T84.': (0, 0, 9, 0),
 'M84.': (1, 0, 5, 0),
 'O10.': (0, 0, 18, 0),
 'K55.': (3, 0, 7, 0),
 'T71.': (1, 0, 1, 0),
 'L29.': (0, 4, 2, 0),
 'Z33.': (1, 73, 3, 0),
 'Z12.': (0, 0, 3, 0),
 'I10.': (7, 31, 217, 0),
 'G90.': (0, 1, 3, 0),
 'D18.': (1, 1, 1, 0),
 'R58.': (1, 151, 0, 0),
 'O41.': (7, 9, 6, 0),
 'W11.': (0, 0, 3, 0),
 'L50.': (0, 2, 1, 0),
 'A54.': (0, 0, 1, 0),
 'R59.': (1, 11, 4, 0),
 'E28.': (0, 0, 3, 0),
 'V48.': (0, 0, 3, 0),
 'G51.': (0, 1, 1, 0),
 'N48.': (2, 0, 1, 0),
 'K74.': (7, 4, 4, 0),
 'R78.': (0, 0, 4, 0),
 'D57.': (5, 2, 4, 0),
 'Z78.': (0, 0, 15, 0),
 'D24.': (0, 2, 2, 0),
 'H47.': (0, 0, 1, 0),
 'Z28.': (0, 0, 2, 0),
 'J20.': (0, 1, 1, 0),
 'G92.': (2, 0, 5, 0),
 'Z80.': (0, 0, 24, 0),
 'T43.': (0, 0, 3, 0),
 'J47.': (0, 0, 2, 0),
 'I65.': (2, 0, 5, 0),

In [75]:



# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall


In [76]:
precisions

{'K13.': 0.0,
 'N11.': 0,
 'N97.': 1.0,
 'S31.': 0.0,
 'D62.': 1.0,
 'L90.': 0.0,
 'K31.': 0.25,
 'C91.': 0,
 'W25.': 0,
 'T84.': 0,
 'M84.': 1.0,
 'O10.': 0,
 'K55.': 1.0,
 'T71.': 1.0,
 'L29.': 0.0,
 'Z33.': 0.013513513513513514,
 'Z12.': 0,
 'I10.': 0.18421052631578946,
 'G90.': 0.0,
 'D18.': 0.5,
 'R58.': 0.006578947368421052,
 'O41.': 0.4375,
 'W11.': 0,
 'L50.': 0.0,
 'A54.': 0,
 'R59.': 0.08333333333333333,
 'E28.': 0,
 'V48.': 0,
 'G51.': 0.0,
 'N48.': 1.0,
 'K74.': 0.6363636363636364,
 'R78.': 0,
 'D57.': 0.7142857142857143,
 'Z78.': 0,
 'D24.': 0.0,
 'H47.': 0,
 'Z28.': 0,
 'J20.': 0.0,
 'G92.': 1.0,
 'Z80.': 0,
 'T43.': 0,
 'J47.': 0,
 'I65.': 1.0,
 'R20.': 0.038461538461538464,
 'H91.': 0.0,
 'D61.': 1.0,
 'C50.': 0.16666666666666666,
 'H80.': 0,
 'N64.': 0.0,
 'S62.': 0.75,
 'Y82.': 0,
 'K08.': 0.0,
 'J11.': 0.25,
 'K58.': 0.5,
 'R00.': 0.3170731707317073,
 'K80.': 0.8181818181818182,
 'X31.': 0,
 'E05.': 0.0,
 'G80.': 0,
 'M18.': 0,
 'O12.': 0,
 'R05.': 0.0384615384615384

In [77]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(precisions.values()) / len(precisions)
macro_recall = sum(recalls.values()) / len(recalls)


In [78]:
macro_precision

0.2656665406872093

In [79]:
macro_recall

0.20711383642839065

In [85]:
# Calculate Micro Precision and Recall
total_TP = sum([counts[icd_code][0] for icd_code in unique_icd_codes])
total_FP = sum([counts[icd_code][1] for icd_code in unique_icd_codes])
total_FN = sum([counts[icd_code][2] for icd_code in unique_icd_codes])

# Handle division by zero
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

Micro Precision: 0.24940898345153664
Micro Recall: 0.15962678098600427


In [86]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score



In [87]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(f1_scores.values()) / len(f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation.

In [88]:
macro_f1_score

0.19709528323555367

In [89]:
# Calculate Micro F1 Score
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

# Handle division by zero for precision and recall in Micro F1 Score
if micro_precision + micro_recall == 0:
    micro_f1_score = 0
else:
    micro_f1_score = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

print("Micro F1 Score:", micro_f1_score)

Micro F1 Score: 0.19466441147074653


In [83]:
# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall

# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in precisions.values() if precision > 0])

In [84]:
average_precision

0.6080284815132856

In [38]:
average_recall = np.mean([recall for recall in recalls.values() if recall > 0])

In [39]:
average_recall

0.4740194649209296

In [40]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score


In [41]:
average_f1 = np.mean([f1_score for f1_score in f1_scores.values() if f1_score > 0])

In [42]:
average_f1

0.4510900976432759

In [43]:
len([f1_score for f1_score in f1_scores.values() if f1_score > 0])

336

#### Top 50 ICD codes identification

In [90]:
top_50_icd_codes = icd_codes_with_smallest_size_notes['icd_code'].value_counts().index[:50]
top_50_icd_codes

Index(['Y92.', 'O99.', 'I10.', 'E78.', 'Z87.', 'Z3A.', 'Z51.', 'Z85.', 'E11.',
       'Z79.', 'Z68.', 'F41.', 'K21.', 'F32.', 'Z66.', 'Z86.', 'E66.', 'I25.',
       'Z37.', 'E87.', 'J45.', 'I48.', 'E03.', 'F17.', 'G47.', 'Y83.', 'Z95.',
       'J96.', 'G93.', 'O26.', 'N17.', 'N18.', 'G89.', 'C78.', 'J44.', 'D64.',
       'R40.', 'O34.', 'Z80.', 'O09.', 'R11.', 'I50.', 'Z23.', 'S06.', 'M19.',
       'N40.', 'I12.', 'I95.', 'S62.', 'O70.'],
      dtype='object')

In [91]:
filtered_test_notes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'].isin(top_50_icd_codes)]

filtered_test_notes

,hadm_id,text,icd_code
3,20019282,Left ankle fracture. ORIF left ankle. yr old m...,Y92.
7,20022343,Preterm labor. repeat low transverse cesarean ...,O34.
9,20022343,Preterm labor. repeat low transverse cesarean ...,Z3A.
10,20022343,Preterm labor. repeat low transverse cesarean ...,Z37.
11,20025548,contractions. SVD. Pt arrives co painful contr...,O99.
...,...,...,...
8578,29990058,induction labor. primary cesarean delivery. Af...,Z3A.
8579,29990058,induction labor. primary cesarean delivery. Af...,Z37.
8581,29990058,induction labor. primary cesarean delivery. Af...,Z23.
8582,29990058,induction labor. primary cesarean delivery. Af...,Z86.


In [92]:
filtered_unique_hadm_notes = filtered_test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')


In [93]:
filtered_unique_hadm_notes = filtered_unique_hadm_notes.reset_index(drop = True)
filtered_unique_hadm_notes

,hadm_id,text
0,20019282,Left ankle fracture. ORIF left ankle. yr old m...
1,20022343,Preterm labor. repeat low transverse cesarean ...
2,20025548,contractions. SVD. Pt arrives co painful contr...
3,20027416,neck cancer. direct laryngoscopy robotic hemig...
4,20028418,admission chemo. Pt tolerated desens without a...
...,...,...
929,29980814,Postpartum incision check. partial opening pac...
930,29980930,elective chemotherapy. Pt presented elective c...
931,29987299,attach. BLOOD WBC RBC Hgb Hct. MCV MCH MCHC RD...
932,29988390,acute cholecystitis. laparoscopic cholecystect...


In [94]:
filtered_icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}



In [95]:
unique_icd_codes = set(filtered_test_notes['icd_code'])

In [96]:
# Step 3: Initialize TP, FP, FN, TN counts
filtered_counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [97]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%10 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = filtered_test_notes[filtered_test_notes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = df_icd10[df_icd10['icd10_codes'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    filtered_counts[icd_code] = (len(TP), len(FP), len(FN), TN)

10 codes are done!
20 codes are done!
30 codes are done!
40 codes are done!
50 codes are done!


In [98]:
filtered_counts

{'O70.': (0, 1, 33, 0),
 'S62.': (3, 1, 14, 0),
 'R11.': (21, 112, 20, 0),
 'Z23.': (0, 0, 40, 0),
 'N17.': (8, 0, 45, 0),
 'Z95.': (1, 2, 47, 0),
 'Z85.': (0, 0, 111, 0),
 'O26.': (0, 0, 49, 0),
 'Y92.': (0, 0, 245, 0),
 'I95.': (10, 10, 24, 0),
 'E78.': (20, 3, 177, 0),
 'J96.': (14, 8, 41, 0),
 'I10.': (7, 31, 217, 0),
 'Z79.': (0, 0, 122, 0),
 'G93.': (20, 10, 22, 0),
 'Z86.': (3, 25, 79, 0),
 'Z37.': (0, 0, 86, 0),
 'G47.': (9, 6, 62, 0),
 'I12.': (0, 0, 34, 0),
 'R40.': (7, 25, 13, 0),
 'M19.': (3, 1, 28, 0),
 'D64.': (6, 11, 39, 0),
 'K21.': (9, 2, 95, 0),
 'I48.': (31, 2, 43, 0),
 'N40.': (3, 0, 31, 0),
 'J44.': (11, 3, 34, 0),
 'O09.': (0, 0, 36, 0),
 'F41.': (15, 17, 91, 0),
 'Z66.': (0, 0, 100, 0),
 'Y83.': (7, 0, 64, 0),
 'G89.': (0, 0, 51, 0),
 'O99.': (1, 0, 129, 0),
 'E87.': (17, 3, 46, 0),
 'Z68.': (0, 0, 109, 0),
 'S06.': (10, 4, 17, 0),
 'J45.': (12, 11, 68, 0),
 'F32.': (12, 8, 88, 0),
 'E66.': (16, 14, 81, 0),
 'Z80.': (0, 0, 24, 0),
 'E03.': (16, 2, 58, 0),
 'Z51.'

In [99]:

# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall

In [100]:
filtered_precisions

{'O70.': 0.0,
 'S62.': 0.75,
 'R11.': 0.15789473684210525,
 'Z23.': 0,
 'N17.': 1.0,
 'Z95.': 0.3333333333333333,
 'Z85.': 0,
 'O26.': 0,
 'Y92.': 0,
 'I95.': 0.5,
 'E78.': 0.8695652173913043,
 'J96.': 0.6363636363636364,
 'I10.': 0.18421052631578946,
 'Z79.': 0,
 'G93.': 0.6666666666666666,
 'Z86.': 0.10714285714285714,
 'Z37.': 0,
 'G47.': 0.6,
 'I12.': 0,
 'R40.': 0.21875,
 'M19.': 0.75,
 'D64.': 0.35294117647058826,
 'K21.': 0.8181818181818182,
 'I48.': 0.9393939393939394,
 'N40.': 1.0,
 'J44.': 0.7857142857142857,
 'O09.': 0,
 'F41.': 0.46875,
 'Z66.': 0,
 'Y83.': 1.0,
 'G89.': 0,
 'O99.': 1.0,
 'E87.': 0.85,
 'Z68.': 0,
 'S06.': 0.7142857142857143,
 'J45.': 0.5217391304347826,
 'F32.': 0.6,
 'E66.': 0.5333333333333333,
 'Z80.': 0,
 'E03.': 0.8888888888888888,
 'Z51.': 0,
 'F17.': 0.0,
 'I50.': 0.7,
 'E11.': 1.0,
 'O34.': 0,
 'Z87.': 0,
 'Z3A.': 0,
 'N18.': 0.9090909090909091,
 'C78.': 0.7777777777777778,
 'I25.': 0.84}

In [101]:
filtered_recalls

{'O70.': 0.0,
 'S62.': 0.17647058823529413,
 'R11.': 0.5121951219512195,
 'Z23.': 0.0,
 'N17.': 0.1509433962264151,
 'Z95.': 0.020833333333333332,
 'Z85.': 0.0,
 'O26.': 0.0,
 'Y92.': 0.0,
 'I95.': 0.29411764705882354,
 'E78.': 0.10152284263959391,
 'J96.': 0.2545454545454545,
 'I10.': 0.03125,
 'Z79.': 0.0,
 'G93.': 0.47619047619047616,
 'Z86.': 0.036585365853658534,
 'Z37.': 0.0,
 'G47.': 0.1267605633802817,
 'I12.': 0.0,
 'R40.': 0.35,
 'M19.': 0.0967741935483871,
 'D64.': 0.13333333333333333,
 'K21.': 0.08653846153846154,
 'I48.': 0.4189189189189189,
 'N40.': 0.08823529411764706,
 'J44.': 0.24444444444444444,
 'O09.': 0.0,
 'F41.': 0.14150943396226415,
 'Z66.': 0.0,
 'Y83.': 0.09859154929577464,
 'G89.': 0.0,
 'O99.': 0.007692307692307693,
 'E87.': 0.2698412698412698,
 'Z68.': 0.0,
 'S06.': 0.37037037037037035,
 'J45.': 0.15,
 'F32.': 0.12,
 'E66.': 0.16494845360824742,
 'Z80.': 0.0,
 'E03.': 0.21621621621621623,
 'Z51.': 0.0,
 'F17.': 0.0,
 'I50.': 0.1794871794871795,
 'E11.': 0.0

In [102]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(filtered_precisions.values()) / len(filtered_precisions)
macro_recall = sum(filtered_recalls.values()) / len(filtered_recalls)


In [103]:
macro_precision

0.42948047895255465

In [104]:
macro_recall

0.12477379459959041

In [105]:
# Calculate Micro Precision and Recall
total_TP = sum([filtered_counts[icd_code][0] for icd_code in unique_icd_codes])
total_FP = sum([filtered_counts[icd_code][1] for icd_code in unique_icd_codes])
total_FN = sum([filtered_counts[icd_code][2] for icd_code in unique_icd_codes])

# Handle division by zero
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

Micro Precision: 0.5178571428571429
Micro Recall: 0.08875286916602908


In [106]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score



In [107]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(filtered_f1_scores.values()) / len(filtered_f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation.

In [108]:
macro_f1_score

0.1700237055570904

In [109]:
# Calculate Micro F1 Score
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

# Handle division by zero for precision and recall in Micro F1 Score
if micro_precision + micro_recall == 0:
    micro_f1_score = 0
else:
    micro_f1_score = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

print("Micro F1 Score:", micro_f1_score)

Micro F1 Score: 0.15153494448073154


In [62]:
# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall    
    
    
    
    
    
# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in filtered_precisions.values() if precision > 0])

In [63]:
average_precision

0.6710632483633665

In [64]:
average_recall = np.mean([recall for recall in filtered_recalls.values() if recall > 0])

In [65]:
average_recall

0.19495905406185993

In [66]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score


In [67]:
average_f1 = np.mean([f1_score for f1_score in filtered_f1_scores.values() if f1_score > 0])

In [68]:
average_f1

0.26566203993295373